In [44]:
import json
from google.oauth2 import service_account
from googleapiclient.discovery import build

# Replace with your service account credentials file
SERVICE_ACCOUNT_FILE = "../env/llmsummer2024-60489508c5f8.json"
user_email = "minling.zhou@gmail.com"
# Scopes required for the API
SCOPES = [
    "https://www.googleapis.com/auth/drive",
    "https://www.googleapis.com/auth/forms.body",
]

# Authenticate and create the service
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES
)
service = build("forms", "v1", credentials=credentials)


def create_form_from_json(json_file):
    # Read JSON file
    with open(json_file, "r") as file:
        questions = json.load(file)

    # Prepare form body
    form_body = {
        "info": {
            "title": f"Google Form from {json_file}",
            "documentTitle": f"Google Form from {json_file}",
        },
    }

    # Create the form
    form = service.forms().create(body=form_body).execute()
    print("Form created: {}".format(form["responderUri"]))

    # share the form
    drive_service = build("drive", "v3", credentials=credentials)

    # Define the permissions
    permissions_body = {"type": "user", "role": "writer", "emailAddress": user_email}

    # Share the form
    drive_service.permissions().create(
        fileId=form["formId"], body=permissions_body, fields="id"
    ).execute()
    print(f"Form shared with {user_email}")

    # Prepare items to be added
    items = []
    for i, question in enumerate(questions):
        question_type = question["type"]
        title = question["title"]
        description = question["description"]
        choices = question.get("choices", [])

        if question_type == "Section":
            question_item = {
                "title": title,
                "description": description,
                "pageBreakItem": {},
            }
        elif question_type == "Dropdown":
            question_item = {
                "title": title,
                "description": description,
                "questionItem": {
                    "question": {
                        "required": True,
                        "choiceQuestion": {
                            "type": "DROP_DOWN",
                            "options": [{"value": choice} for choice in choices],
                        },
                    }
                },
            }
        elif question_type == "Multiple Choice":
            question_item = {
                "title": title,
                "description": description,
                "questionItem": {
                    "question": {
                        "required": True,
                        "choiceQuestion": {
                            "type": "RADIO",
                            "options": [{"value": choice} for choice in choices],
                        },
                    }
                },
            }
        elif question_type == "Text":
            question_item = {
                "title": title,
                "description": description,
                "questionItem": {"question": {"required": True, "textQuestion": {}}},
            }
        elif question_type == "Date":
            question_item = {
                "title": title,
                "description": description,
                "questionItem": {"question": {"required": True, "dateQuestion": {}}},
            }

        items.append({"createItem": {"item": question_item, "location": {"index": i}}})

    # Update the form
    service.forms().batchUpdate(
        formId=form["formId"],
        body={"requests": items},
    ).execute()

# Generate Google Form from JSON file

In [45]:
# Path to your JSON file
json_file_path = "GF_LLM_2024-05-19_eng.json"
create_form_from_json(json_file_path)

Form created: https://docs.google.com/forms/d/e/1FAIpQLSccUe0PJRC5iLwhherB80XIeYxmnckm5jYKKqVxnNfKnQsZLQ/viewform
Form shared with minling.zhou@gmail.com


In [46]:
# Path to your JSON file
json_file_path = "GF_LLM_2024-05-19_hin.json"
create_form_from_json(json_file_path)

Form created: https://docs.google.com/forms/d/e/1FAIpQLSc4IXHonfSwda27_tU6hmPURMaDK1puxfwFttJzF4XXB8OELQ/viewform
Form shared with minling.zhou@gmail.com


In [47]:
# Path to your JSON file
json_file_path = "GF_LLM_2024-05-19_zho.json"
create_form_from_json(json_file_path)

Form created: https://docs.google.com/forms/d/e/1FAIpQLSdcXESV6Ev0_-G9HCL2ySNXsBmdH0eEbxEiCIGqVli7rPi2wA/viewform
Form shared with minling.zhou@gmail.com


# Share the google form

In [20]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
import os

# Share the form with a specific email address
user_email = "minling.zhou@gmail.com"  # Replace with your email address
form_id = "1D512HrppJIac82RkVruUNojmioTuMryu1fjA6JbIUq8"
SCOPES = [
    "https://www.googleapis.com/auth/drive",
    "https://www.googleapis.com/auth/forms.body",
]

credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES
)

drive_service = build("drive", "v3", credentials=credentials)
# Define the permissions
permissions_body = {"type": "user", "role": "writer", "emailAddress": user_email}

# Share the form
drive_service.permissions().create(
    fileId=form_id, body=permissions_body, fields="id"
).execute()
print(f"Form shared with {user_email}")

Form shared with minling.zhou@gmail.com


# List google form as the owner

In [42]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

# Replace with your service account credentials file
SERVICE_ACCOUNT_FILE = "../env/llmsummer2024-60489508c5f8.json"

# Scopes required for the API
SCOPES = ["https://www.googleapis.com/auth/drive.metadata.readonly"]

# Authenticate and create the service
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES
)
service = build("drive", "v3", credentials=credentials)


def list_google_forms():
    # Query to find all Google Forms
    query = "mimeType='application/vnd.google-apps.form'"

    results = (
        service.files().list(q=query, pageSize=100, fields="files(id, name)").execute()
    )
    items = results.get("files", [])

    if not items:
        print("No Google Forms found.")
    else:
        print("Google Forms:")
        for item in items:
            print(f"{item['name']} (ID: {item['id']})")


# List all Google Forms
list_google_forms()

No Google Forms found.


# Delete the google form by form id

In [41]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

# Path to your service account credentials file
# Replace with your service account credentials file
SERVICE_ACCOUNT_FILE = "../env/llmsummer2024-60489508c5f8.json"

# Scopes required for the API
SCOPES = ["https://www.googleapis.com/auth/drive"]

# Authenticate and create the service
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES
)
drive_service = build("drive", "v3", credentials=credentials)


def delete_google_form(form_id):
    try:
        # Delete the form
        drive_service.files().delete(fileId=form_id).execute()
        print(f"Form with ID {form_id} has been deleted successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")


# The ID of the form you want to delete
form_id = (
    "1HDZIlaPmfDi1bpA-0MHDr8IDlcYZY-0O3E8Q4SYZyqU"  # Replace with your Google Form ID
)

# Call the function to delete the form
delete_google_form(form_id)

Form with ID 1HDZIlaPmfDi1bpA-0MHDr8IDlcYZY-0O3E8Q4SYZyqU has been deleted successfully.
